In [1]:
!git clone https://github.com/Dapur-Nusantara-Bangkit-2024-Batch-2/machine-learning.git

Cloning into 'machine-learning'...
remote: Enumerating objects: 1211, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (226/226), done.
remote: Total 1211 (delta 62), reused 6 (delta 3), pack-reused 982
Receiving objects: 100% (1211/1211), 44.82 MiB | 30.39 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import warnings

warnings.filterwarnings("ignore")

# Define paths
train_dir = '/content/machine-learning/dataset/train'
test_dir = '/content/machine-learning/dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=batch_size,
                                             image_size=(img_height, img_width))

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=batch_size,
                                            image_size=(img_height, img_width))

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width, 3),
                                               include_top=False,
                                               weights='imagenet')

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Add classification head
num_classes = len(train_dataset.class_names)

model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])



# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Callbacks for early stopping and model checkpoint
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 10
history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=epochs,
                    callbacks=[early_stopping, model_checkpoint, reduce_lr])

# Load the best model
model.load_weights('best_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as 'model.tflite'")


Found 793 files belonging to 20 classes.
Found 199 files belonging to 20 classes.
9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
25/25 [==============================] - 66s 2s/step - loss: 2.5743 - accuracy: 0.2320 - val_loss: 2.3216 - val_accuracy: 0.3015 - lr: 1.0000e-04
Epoch 2/10
25/25 [==============================] - 60s 2s/step - loss: 1.3865 - accuracy: 0.6797 - val_loss: 1.7724 - val_accuracy: 0.4472 - lr: 1.0000e-04
Epoch 3/10
25/25 [==============================] - 57s 2s/step - loss: 0.8753 - accuracy: 0.8146 - val_loss: 1.3891 - val_accuracy: 0.5879 - lr: 1.0000e-04
Epoch 4/10
25/25 [==============================] - 58s 2s/step - loss: 0.5842 - accuracy: 0.8852 - val_loss: 1.1782 - val_accuracy: 0.6482 - lr: 1.0000e-04
Epoch 5/10
25/25 [==============================] - 57s 2s/step - loss: 0.4679 - accuracy: 0.9067 - val_loss: 1.0787 - val_accuracy: 0.6784 - lr: 1.0000e-04
Epoch 6/10
25/25 [==============================] - 62s 2s/step - loss